In [59]:
!nvidia-smi

Thu Apr 23 05:38:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    43W / 250W |   6111MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install kaggle

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
!kaggle datasets download -d greg115/celebrities-100k

 99% 820M/828M [00:12<00:00, 30.0MB/s]
100% 828M/828M [00:12<00:00, 69.4MB/s]


In [ ]:
!unzip -q celebrities-100k.zip

In [0]:
!rm celebrities-100k.zip

In [9]:
!python -m pip install git+https://github.com/ShivamShrirao/dnn_from_scratch.git

Cloning into 'dnn_from_scratch'...
remote: Enumerating objects: 330, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1021 (delta 231), reused 192 (delta 98), pack-reused 691
Receiving objects: 100% (1021/1021), 12.97 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (656/656), done.


In [0]:
import timeit
import matplotlib.pyplot as plt
import pickle
from time import time
import numpy as np

In [0]:
from matplotlib.image import imread
import cv2

In [0]:
DATA_DIR="/content/100k"
with open(DATA_DIR+".txt","r") as f:
    names=f.read().split('\n')[:-1]

In [5]:
from nnet_gpu.network import Sequential
from nnet_gpu.layers import Conv2D,MaxPool,Flatten,Dense,Dropout,Conv2Dtranspose,Upsampling,Reshape,BatchNormalization,Activation
from nnet_gpu import functions
from nnet_gpu import optimizers
import numpy as np
import cupy as cp

In [6]:
def generator():
    model=Sequential()
    model.add(Dense(8*8*512,activation=functions.leakyRelu,input_shape=100))
    model.add(Reshape((8,8,512)))
    model.add(Upsampling())
    model.add(Conv2D(512,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Upsampling())
    model.add(Conv2D(256,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Upsampling())
    model.add(Conv2D(128,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Upsampling())
    model.add(Conv2D(64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Conv2D(3,kernel_size=3,activation=functions.tanh))
    return model
g=generator()
g.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 100)                echo             0
__________________________________________________________________________________________
0 Dense(Dense)            (None, 32768)              leakyRelu        3309568
__________________________________________________________________________________________
1 Reshape(Reshape)        (None, 8, 8, 512)          echo             0
__________________________________________________________________________________________
2 Upsampling(Upsampling)  (None, 16, 16, 512)        echo             0
__________________________________________________________________________________________
3 Conv2D(Conv2D)          (None, 16, 16, 512)        echo             2359808
__________________________________________________________________________________________
4 Ba

In [7]:
def discriminator():
    model=Sequential()
    model.add(Conv2D(64,kernel_size=3,stride=[2,2],input_shape=(128,128,3)))
    model.add(Activation(functions.leakyRelu))
    model.add(Conv2D(128,kernel_size=3,stride=[2,2]))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Conv2D(256,kernel_size=3,stride=[2,2]))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Conv2D(512,kernel_size=3,stride=[2,2]))
    model.add(BatchNormalization())
    model.add(Activation(functions.leakyRelu))
    model.add(Flatten())
    model.add(Dense(1,activation=functions.sigmoid))
    return model
d=discriminator()
d.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 128, 128, 3)        echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 64, 64, 64)         echo             1792
__________________________________________________________________________________________
1 Activation(Activation)  (None, 64, 64, 64)         leakyRelu        0
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 32, 32, 128)        echo             73856
__________________________________________________________________________________________
3 BatchNormalization(Batc (None, 32, 32, 128)        echo             512
__________________________________________________________________________________________
4 Activ

In [0]:
g.compile(optimizer=optimizers.adam,loss=functions.mean_squared_error,learning_rate=0.0004,beta=0.5)
d.compile(optimizer=optimizers.adam,loss=functions.cross_entropy,learning_rate=0.0004,beta=0.5)

In [0]:
svdnoise=cp.random.randn(16,100).astype(cp.float32)
gen=g.predict(svdnoise)

In [0]:
def gen_sample():
    noise=cp.random.randn(1,100).astype(cp.float32)
    gen=g.predict(noise)
    plt.imshow((gen[0].get()+1)/2,interpolation='bicubic')
    plt.show()
    print(d.predict(gen)[0])

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def gen_grid(noise,show=True,save=False):
    fig, ax = plt.subplots(nrows=4,ncols=4,figsize=[20,20],gridspec_kw = {'wspace':0, 'hspace':0})
    fig.patch.set_facecolor('white')
    gen=g.predict(noise).get()
    for i,axi in enumerate(ax.flat):
        axi.axis("off")
        axi.imshow((gen[i].squeeze()+1)/2,interpolation='bicubic')
    if save:
        fig.savefig("/content/drive/My Drive/timelapse/"+str(ccu)+"img"+str(time())[-5:]+".png",bbox_inches='tight')
    if not show:
        plt.close(fig)

In [0]:
ccu=0

In [0]:
batch_size=32

In [0]:
from threading import Thread

In [0]:
transfer_stream=cp.cuda.stream.Stream(non_blocking=True)
default_stream=cp.cuda.get_current_stream()

In [0]:
bch_img=None
event=transfer_stream.record()

In [0]:
def get_batch(st,btsz):
    X_train=[]
    for ii in names[st:st+btsz]:
        img=imread(DATA_DIR+"/100k/"+ii)
#         img=cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_AREA)
        X_train.append(img/255*2-1)
    global bch_img,event#,transfer_stream
    # with transfer_stream:
    bch_img=cp.asarray(X_train,dtype=np.float32)
    # event=transfer_stream.record(event)

In [0]:
t=Thread(target=get_batch,args=(ccu,batch_size,))
t.setDaemon(True)
t.start()

In [0]:
def run(batch_size=16):
    global ccu,svdnoise
    dsz=len(names)
    # Label real ones as 90% real
    y_fake=cp.zeros((batch_size,1),dtype=cp.float32)
    y_real=cp.ones((batch_size,1),dtype=cp.float32)
    y_gen=cp.ones((batch_size,1),dtype=cp.float32)
    elap=time()
    for step in range(dsz//batch_size):
        # Get random real images
        global bch_img,event,default_stream,t
        t.join()
        # default_stream.wait_event(event)
        real_images=bch_img
        t=Thread(target=get_batch,args=(ccu,batch_size,))
        t.setDaemon(True)
        t.start()
        if len(real_images)!=batch_size:
          continue
        ccu+=batch_size
        if ccu>len(names):
            ccu=0
            break
        stt=time()
        noise=cp.random.randn(batch_size,100,dtype=cp.float32)
        # Generate fake images from noise
        generated_images=g.predict(noise)
        # Train discriminator
        y_fake=cp.random.uniform(0,0.05,(batch_size,1),dtype=cp.float32)
        y_real=cp.random.uniform(0.95,1,(batch_size,1),dtype=cp.float32)
        dout1=d.train_on_batch(real_images,y_real)
        dout2=d.train_on_batch(generated_images,y_fake)
        dloss=functions.cross_entropy(logits=dout2,labels=y_fake).sum()
        # Treat noised input of generator as real data
        noise=cp.random.randn(batch_size,100,dtype=cp.float32)
        # Train generator
        gout=g.forward(noise)
        # do not train discriminator and find delta for generator
        dout,err=d.not_train_on_batch(gout,y_gen)
        # backpropogate the generator and update weights
        g.backprop(err,g.lenseq_m1)
        g.optimizer(g.sequence,g.learning_rate,g.beta)
        stptm=time()-stt
        if not step%120 and step>0:
            print("\nSaving grid.")
            gen_grid(svdnoise[:25],show=False,save=False)
            gen_sample()
        # global transfer_stream
        # with transfer_stream:
        print(f"\rProgress: {step*batch_size*100/dsz:.2f} % - {ccu} - {stptm:.3f}s/step - Elapsed: {time()-elap:.6}s - Dloss: {dloss.get():.6} - err: {abs(err).sum().get():.6}    _",end='')
    if ccu>=len(names):
        ccu=0

In [0]:
import gc

In [0]:
gc.collect()
cp.get_default_pinned_memory_pool().free_all_blocks()
cp.get_default_memory_pool().free_all_blocks()
# ccu=0

In [0]:
epochs=5
for epoch in range(epochs):
    gc.collect()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    cp.get_default_memory_pool().free_all_blocks()
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run(batch_size)
    print("\nEpoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))
    g.save_weights("/content/drive/My Drive/generator.w8s")
    d.save_weights("/content/drive/My Drive/discriminator.w8s")

EPOCH: 1 / 5
Progress: 0.74 % - 768 - 0.245s/step - Elapsed: 23.8579s - Dloss: 14.3768 - err: 1860.72    _

## # Using generator with just batch size 1 for first time is causing images to be generated extreme

In [0]:
noise=cp.random.randn(16,100).astype(cp.float32)
gen=g.predict(noise)
gen_sample()
imdt=[]
st=np.random.randint(low=0,high=len(names))
for ii in names[st:st+1]:
    img=imread(DATA_DIR+"/100k/"+ii)
#         img=cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_AREA)
    imdt.append(img/255*2-1)
img=cp.asarray(imdt)
plt.imshow((img[0].get()+1)/2,interpolation='bicubic')
plt.show()
print(d.predict(img))

In [0]:
noise=cp.random.randn(16,100).astype(cp.float32)
gen_grid(noise)

In [0]:
# g.load_weights("/content/drive/My Drive/generator.w8s")
# d.load_weights("/content/drive/My Drive/discriminator.w8s")

In [0]:
g.save_weights("/content/drive/My Drive/generator.w8s")
d.save_weights("/content/drive/My Drive/discriminator.w8s")